In [ ]:
%pylab notebook

#### Must pip install astropy, lightkurve and PyAstronomy.pyasl

In [4]:
from astropy.table import Table
from astropy.time import Time
from astropy.io import ascii
import lightkurve as lk 
import warnings 
from lightkurve import search_targetpixelfile
import pandas as pd
from astropy import units as u


###### uncomment the below if not using magic: ######
# import matplotlib.pyplot as plt
# import numpy as np

#### Searching and retrieving the light curve(s) of a star:

In [ ]:
#######EDITABLE#######
#Insert the name of the star of interest below inbetween the red quotes.
star_name = '' #what the catalogue contains
file_name = '' #must contain NO spaces or file extension
Gaia_file_name = '' #give Gaia FITS file (without extension)

In [ ]:
#Finding star
search_result = lk.search_lightcurve(star_name, mission = 'TESS')

In [ ]:
#Showing available data
search_result

In [ ]:
#######EDITABLE#######
#Enter what parameters to narrow down data downloaded
#Specify in ''!!!

exptime = 120 # specify the exposure time (s) - just like cadance

quarter = None # specify the sector

author = 'SPOC' # specify the author

radius=None # default is 0.0001 arcsec (conesearch)

limit=None # maximum number of products to return

In [ ]:
search_result = lk.search_lightcurve(star_name, mission = 'TESS',
                                     exptime=exptime,
                                     quarter=quarter,
                                     author= author,
                                     radius=radius,
                                     limit=limit)

print(search_result)

In [ ]:
#Downloading filtered data
lc = search_result[0].download_all()
count_result = len(search_result) #number of search results

In [ ]:
#Displaying the data
for lightcurve in range(0,count_result):
    current = lc[lightcurve]
    current.plot(label='Result No. '+str(lightcurve),color='orange')
    plt.show()

In [ ]:
#######EDITABLE#######
#Enter data that is not of interest 
#(i.e. those which, by eye, don't look to be of good quality)
unwanted = []

In [ ]:
#########NOT EDITABLE: Only run if wanting to remove whole search results#########
#Showing and downloading the newly filtered results
wanted = []

for lightcurve in range(0,count_result):
    if lightcurve not in unwanted:
        wanted.append(lightcurve)

print(wanted)

search_result = search_result[wanted]
print(search_result) #printing updated list
lc = search_result.download_all() #downloading wanted lightcurves

In [ ]:
#Displaying light curve<s> of interest:
nlc = lc.stitch()
nlc.plot(color='purple')
plt.show()

#### Removing the messy bits

In [ ]:
#######EDITABLE#######
#Examine the light curve(s) of interest above using the interactive plot tool.
#Insert the start and end time(s) of the unwanted patch(es) of data into the following variables respectively.
bad_dat_start_list = [] #The start times of individual unwanted patches are separated by ','.
bad_dat_end_list =  []  #The end times of individual unwanted patches are separated by ','.

bad_flux_bottom = [] #bottom (y values) of bad values
bad_flux_top = [] # top (y values) of bad values

In [ ]:
good_lc = nlc.copy()
for bad_dat_start,bad_dat_end in zip(bad_dat_start_list,bad_dat_end_list):
    print('Removing data from ',bad_dat_start,' to ',bad_dat_end)
    #Extracting and displaying the good data
    good_data_mask =  (good_lc.time.value < bad_dat_start) | (good_lc.time.value > bad_dat_end)
    good_lc = good_lc[good_data_mask]

for bad_dat_start,bad_dat_end in zip(bad_flux_bottom,bad_flux_top):
    print('Removing data from ',bad_dat_start,' to ',bad_dat_end)
    #Extracting and displaying the good data
    good_data_mask =  (good_lc.flux.value < bad_dat_start) | (good_lc.flux.value > bad_dat_end)
    good_lc = good_lc[good_data_mask]

In [ ]:
#Plotting the good data
fig, ax = plt.subplots(figsize=(10,5))
good_lc.scatter(ax=ax, s=1, color='dodgerblue')
ax.set_title('The Good Stuff')    
#ax.set_xlim(0,1700)

#### Determining the orbital period of the star:

In [ ]:
#######EDIT THIS#######

#Use the interactive plot tool to measure an estimate of the orbital period of the star.
#Insert this estimate along with a reasonable uncertainty in the corresponding variables below.


period_estimate = None #estimate in days
period_uncertainty = 0.1 #uncertainty in days

In [ ]:
#Creating an array of periods to search
period = np.linspace(period_estimate-period_uncertainty, period_estimate+period_uncertainty, 10000)

#Creating a BLSPeriodogram
bls = good_lc.to_periodogram(method='bls', period=period, frequency_factor=500);
bls.plot(color='firebrick');

In [ ]:
orbital_period = bls.period_at_max_power
orbital_t0 = bls.transit_time_at_max_power
orbital_dur = bls.duration_at_max_power

#Displaying the most likely orbital period of the star
print(orbital_period)
print(orbital_t0)

In [ ]:
#Plotting a phase-folded light curve of the star
ax = good_lc.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period).scatter(color='mediumseagreen')
plt.show()

#### Comparing with raw Gaia data

In [ ]:
#This is to import the .fits file with the gaia data and then getting the data into the 3 different colour bands for comparison
#with the phase folded lightcurve above.

table = Table.read(str(Gaia_file_name)+'.fits')

u.add_enabled_units(u.def_unit('electrons.s**-1'))

#Creating LightCurve objects from the data for each band (using flux directly)
Glc = lk.LightCurve(time=table['g_transit_time'].value + 2455197.5,
                    flux=table['g_transit_flux'].value,
                    flux_err=table['g_transit_flux_error'].value)
Glc['time'].format='btjd'
 
BP_lc = lk.LightCurve(time=table['bp_obs_time'].value + 2455197.5,
                    flux=table['bp_flux'].value,
                    flux_err=table['bp_flux_error'].value)
BP_lc['time'].format='btjd'
 
RP_lc = lk.LightCurve(time=table['rp_obs_time'].value + 2455197.5,
                    flux=table['rp_flux'].value,
                    flux_err=table['rp_flux_error'].value)
RP_lc['time'].format='btjd'



In [ ]:
print("Green Band NaNs:", np.isnan(table['g_transit_flux']).sum())
print("Blue Band NaNs:", np.isnan(table['bp_flux']).sum())
print("Red Band NaNs:", np.isnan(table['rp_flux']).sum())



In [ ]:
#Normalising the Gaia data.

nGlc = Glc.normalize()

nBP_lc = BP_lc.normalize()

nRP_lc = RP_lc.normalize()

In [ ]:
print(" normalised Green Band NaNs:", np.isnan(nGlc['flux']).sum())
print(" normalised Blue Band NaNs:", np.isnan(nBP_lc['flux']).sum())
print("normalised Red Band NaNs:", np.isnan(nRP_lc['flux']).sum())

#### Only clean the bands with nan values - if they don't contain nan values then DO NOT run and move on to phase folding



In [ ]:
nGlc_clean = nBP_lc[~np.isnan(nBP_lc['flux'])]

In [ ]:
nBP_lc_clean = nBP_lc[~np.isnan(nBP_lc['flux'])]

In [ ]:
nRP_lc_clean = nRP_lc[~np.isnan(nRP_lc['flux'])]

#### Phase folding the Gaia data and the good_lc to prepare for overplotting:

##### Green band - use the one you need; simply get rid of (#)


In [ ]:
#nGlc_pf = nGlc.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period)


#nGlc_pf = nGlc_clean.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period

##### Red band - use the one you need; simply get rid of (#)

In [ ]:
#nRP_lc_pf = nRP_lc.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period)

#nRP_lc_pf = nRP_lc_clean.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period)

##### Blue band - use the one you need; simply get rid of (#)


In [ ]:
#nBP_lc_pf = nBP_lc.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period)

#nBP_lc_pf = nBP_lc_clean.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period)

In [ ]:
#RUN this cell!!

lc_after_pf = good_lc.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period) 

In [ ]:
print("phase folded Green Band Time Range:", np.min(nGlc_pf.time), np.max(nGlc_pf.time))
print("phase folded Blue Band Time Range:", np.min(nBP_lc_pf.time), np.max(nBP_lc_pf.time))
print("phase folded Red Band Time Range:", np.min(nRP_lc_pf.time), np.max(nRP_lc_pf.time))

In [ ]:
#Overplotting the Gaia data with the phase folded lightcurve: if they look similar then the data is good to use.

fig,ax = plt.subplots()
lc_after_pf.scatter(ax = ax, c = 'wheat',marker='+')
nRP_lc_pf.scatter(ax = ax, c= 'r',marker='o')
nGlc_pf.scatter(ax = ax, c = 'g',marker='o')
nBP_lc_pf.scatter(ax = ax, c = 'b',marker='o')
plt.show()

#### Detrending the light curve
- Change `polynomial_degree` to smaller if **poor fit**
- Change `gap_tol` if **error of no x values in array**

Note, this section (detrending the light curve) is adapted from code written by Pierre Maxted (2024), sent in private communications.

In [ ]:
#####EDITABLE#####
#Adjust some of the parameters on the first three lines to get a good fit to your light curve.

n_cycle = 5             # recommended value = 2 - 5
rejection_limit = 4     # recommended value = 3 - 5
polynomial_degree = 12  # recommended value = 2 - 16, do not use very high values (>~24)
gap_tol = 0.05       # recommended value = 0.005, smaller value => more gaps

In [ ]:
#No user servicable parts inside this chunk of code
t =  good_lc.time.value
lc_sorted = good_lc[np.argsort(t)]
timeval =  lc_sorted.time.value
start_times = np.hstack([timeval.min(),timeval[np.gradient(timeval)>gap_tol][1::2]])
end_times = np.hstack([timeval[np.gradient(timeval)>gap_tol][0::2],timeval.max()])
smooth_function = np.ones_like(timeval)
plot_function = np.zeros_like(timeval) + np.nan
time_fit = np.array([])
flux_fit = np.array([])
for i,(t_min,t_max) in enumerate(zip(start_times,end_times)):
    j = (timeval >= t_min) & (timeval <= t_max)
    t_ref = np.median(timeval[j])
    x = timeval[j] - t_ref
    y = lc_sorted.flux.value[j]
    ft = np.ones_like(x) 
    if sum(j) > (polynomial_degree*2):
        for cycle in range(n_cycle):
            r = y/ft
            fit_mask = np.abs(r-1) < rejection_limit*np.mean(np.abs(r-1))
            x = x[fit_mask]
            y = y[fit_mask]
            poly_coeffs = np.polyfit(x, y, polynomial_degree)
            ft = np.polyval(poly_coeffs, x)
    time_fit = np.hstack([time_fit,x+t_ref])
    flux_fit = np.hstack([flux_fit,y])
    smooth_function[j] = np.polyval(poly_coeffs, timeval[j] - np.median(timeval[j]))
    # By using "<" in the second condition we leave an np.nan value in the array
    # at the gap position, so we get a gap in the plotted function.
    j = (timeval >= t_min) & (timeval < t_max)
    plot_function[j] =  np.polyval(poly_coeffs, timeval[j] - np.median(timeval[j]))
lc_after = lc_sorted / smooth_function

lc_after_1 = lc_after

##### Plot fit to trends in the data
 - Black points are data included in the polynomial fit
 - Grey points are points ignored in the polynomial fit
 - Red lines are the polynomials fits to each chunk of data

In [ ]:
#Adjust figure size here, if needed
 
year = np.round((timeval-timeval.min())/365.25)
n_year = len(set(year))
fig,axlist = plt.subplots(ncols=n_year,figsize=(10,4),
                gridspec_kw = {'wspace':0.01},sharey=True)
 
for i,y in enumerate(set(year)):
    if n_year > 1:
        ax = axlist[i]
    else:
        ax = axlist
    j = year == y
    ax.scatter(timeval[j],good_lc.flux[j],s=1,color='grey')
    ax.scatter(time_fit,flux_fit,s=2,color='navy')
    ax.plot(timeval[j],plot_function[j],c='r')
    ax.set_xlabel('Time  (days)')
    if i == 0:
        r = np.max([2*np.ptp(smooth_function),3*np.std(flux_fit)])
        ax.set_ylim(1-r,1+r)
        ax.set_ylabel('Normalised flux')
    ax.set_xlim(timeval[j].min(),timeval[j].max())
    for t in (start_times[1:]+end_times[:-1])/2:
        ax.axvline(t,c='k',ls=':')
plt.show()

#### Remove points where the red curve is not flat (unconstrained)
- Change the polynomial fit if grey dots form
- Remove the data from that point if the red line suddenly jumps (as below)

In [ ]:
#######EDITABLE#######
#Examine the light curve(s) of interest above using the interactive plot tool.
#Insert the start and end time(s) of the unwanted patch(es) of data into the following variables respectively.
bad_dat_start_list_2 = [] #The start times of individual unwanted patches are separated by ','.
bad_dat_end_list_2 =  []  #The end times of individual unwanted patches are separated by ','.

In [ ]:
lc_after = lc_after.copy()
for bad_dat_start,bad_dat_end in zip(bad_dat_start_list_2,bad_dat_end_list_2):
    print('Removing data from ',bad_dat_start,' to ',bad_dat_end)
    #Extracting and displaying the good data
    good_data_mask =  (lc_after.time.value < bad_dat_start) | (lc_after.time.value > bad_dat_end)
    lc_after = lc_after[good_data_mask]

In [ ]:
#Plotting the good data
fig, ax = plt.subplots(figsize=(10,5))
lc_after.scatter(ax=ax, s=1, color='dodgerblue')
ax.set_title('The Good Stuff')    
#ax.set_xlim(0,1700)

#### Re-do Detrending

In [ ]:
#No user serviceable parts inside this chunk of code
lc_after = lc_after.remove_nans() #removing nan values

t =  lc_after.time.value
lc_sorted1 = lc_after[np.argsort(t)]
timeval =  lc_sorted1.time.value
start_times = np.hstack([timeval.min(),timeval[np.gradient(timeval)>gap_tol][1::2]])
end_times = np.hstack([timeval[np.gradient(timeval)>gap_tol][0::2],timeval.max()])
smooth_function = np.ones_like(timeval)
plot_function = np.zeros_like(timeval) + np.nan
time_fit = np.array([])
flux_fit = np.array([])
for i,(t_min,t_max) in enumerate(zip(start_times,end_times)):
    j = (timeval >= t_min) & (timeval <= t_max)
    t_ref = np.median(timeval[j])
    x = timeval[j] - t_ref
    y = lc_sorted1.flux.value[j]
    ft = np.ones_like(x) 
    if sum(j) > (polynomial_degree*2):
        for cycle in range(n_cycle):
            r = y/ft
            fit_mask = np.abs(r-1) < rejection_limit*np.mean(np.abs(r-1))
            x = x[fit_mask]
            y = y[fit_mask]
            poly_coeffs = np.polyfit(x, y, polynomial_degree)
            ft = np.polyval(poly_coeffs, x)
    time_fit = np.hstack([time_fit,x+t_ref])
    flux_fit = np.hstack([flux_fit,y])
    smooth_function[j] = np.polyval(poly_coeffs, timeval[j] - np.median(timeval[j]))
    # By using "<" in the second condition we leave an np.nan value in the array
    # at the gap position, so we get a gap in the plotted function.
    j = (timeval >= t_min) & (timeval < t_max)
    plot_function[j] =  np.polyval(poly_coeffs, timeval[j] - np.median(timeval[j]))
lc_after = lc_sorted1 / smooth_function

##### Plot fit to trends in the data
 - Black points are data included in the polynomial fit
 - Grey points are points ignored in the polynomial fit
 - Red lines are the polynomials fits to each chunk of data

In [ ]:
#Adjust figure size here, if needed
#Comparing ORIGINAL (before any removal of trends) to second trend removal
 
year = np.round((timeval-timeval.min())/365.25)
n_year = len(set(year))
fig,axlist = plt.subplots(ncols=n_year,figsize=(10,4),
                gridspec_kw = {'wspace':0.01},sharey=True)
 
for i,y in enumerate(set(year)):
    if n_year > 1:
        ax = axlist[i]
    else:
        ax = axlist
    j = year == y
    ax.scatter(timeval[j],lc_after.flux[j],s=1,color='grey')
    ax.scatter(time_fit,flux_fit,s=2,color='navy')
    ax.plot(timeval[j],plot_function[j],c='r')
    ax.set_xlabel('Time  (days)')
    if i == 0:
        r = np.max([2*np.ptp(smooth_function),3*np.std(flux_fit)])
        ax.set_ylim(1-r,1+r)
        ax.set_ylabel('Normalised flux')
    ax.set_xlim(timeval[j].min(),timeval[j].max())
    for t in (start_times[1:]+end_times[:-1])/2:
        ax.axvline(t,c='k',ls=':')

 
#Uncomment this line and set file name if you want a copy of the figure for your report
#fig.savefig('trend_removal.png',dpi=144)

#### Compare light curves before and after trend removal
**N.B. The graph here shows the improvement upon the previous detrending NOT the original**

In [ ]:
fig,axlist = plt.subplots(ncols=n_year,figsize=(14,4),
                gridspec_kw = {'wspace':0.01},sharey=True)
for i,y in enumerate(set(year)):
    ax = axlist[i]
    j = year == y
    if i == 0:
        lc_sorted1[j].scatter(ax=ax, s=1,c='r',label='Before')
        lc_after[j].scatter(ax=ax,s=1,c='b',label='After')
    else:
        lc_sorted1[j].scatter(ax=ax, s=1,c='r',label=None)
        lc_after[j].scatter(ax=ax,s=1,c='b',label=None)
        ax.set_ylabel(None)

#### Phase fold light curve again

In [ ]:
#Creating an array of periods to search
period = np.linspace(period_estimate-period_uncertainty, period_estimate+period_uncertainty, 10000)

#Creating a BLSPeriodogram
bls = lc_after.to_periodogram(method='bls', period=period, frequency_factor=500);
bls.plot(color='firebrick');

In [ ]:
orbital_period = bls.period_at_max_power
orbital_t0 = bls.transit_time_at_max_power
orbital_dur = bls.duration_at_max_power

#Displaying the most likely orbital period of the star
print(orbital_period)
print(orbital_t0)

In [ ]:
#Plotting a phase-folded light curve of the star
ax = lc_after.fold(period=orbital_period, epoch_time=orbital_t0,wrap_phase=0.8*orbital_period).scatter(color='mediumseagreen')
ax.set_xlim(-2.5,5)
plt.show()

#### 

#### Did the trend removal work? 
##### Yes: Run the cell below
##### No: SKIP the cell below. Continue to next section

In [ ]:
#ONLY run if happy with trend removal.
#If not, either:
# (a) skip this cell 
# (b) return to top of trend removal sequence to try the process again.
good_lc = lc_after

#### Create Mask for Eclipses
Look at the **phase folded lightcurve** and input values below for:
- Width of primary eclipse - this **must** include some flat area around the eclipse
- Width of secondary eclipse (or `None` if none)
- Phase of secondary eclipse (or `None` if none) - this is of the centre of the dip

All must be phases

In [ ]:
primary_eclipse_width = None #width of primary eclipse
secondary_eclipse_width = None #width of secondary eclipse (if none, write `None`)
secondary_eclipse_phase_relative = None #phase of secondary eclipse (if none, write `None`), as seen in phase folded lightcurve x-coordinate

In [ ]:
orbital_t0_bin = orbital_t0.value
orbital_period_bin = orbital_period.value

secondary_eclipse_phase = secondary_eclipse_phase_relative * (1/orbital_period_bin)

start_primary0 = orbital_period_bin - (primary_eclipse_width)/2 #start of first eclipse
end_primary0 = orbital_period_bin + (primary_eclipse_width)/2 #end of first eclipse

end_sequence = max(good_lc.time.value) #end time 

num_primary_eclipses = int(np.floor((end_sequence - end_primary0)/orbital_period_bin)) #number of primary eclipses

start_primary = []
end_primary = []
for i in range(0,num_primary_eclipses):
    eclipse_start = ((orbital_period_bin)+(orbital_period_bin*i))-(primary_eclipse_width)/2 
    eclipse_end = ((orbital_period_bin)+(orbital_period_bin*i))+(primary_eclipse_width)/2
    start_primary.append(eclipse_start)
    end_primary.append(eclipse_end)
mask_limits = np.column_stack([start_primary,end_primary])

if secondary_eclipse_width is not None:
    start_secondary0 = (orbital_t0_bin+secondary_eclipse_phase) - (secondary_eclipse_width)/2 #start of first eclipse
    end_secondary0 = (orbital_t0_bin+secondary_eclipse_phase) + (secondary_eclipse_width)/2 #end of first eclipse
    num_secondary_eclipses = int(np.floor((end_sequence - end_secondary0)/orbital_period_bin)) #number of secondary eclipses

    start_secondary = []
    end_secondary = []
    for i in range(0,num_secondary_eclipses):
        eclipse_start = ((orbital_t0_bin+secondary_eclipse_phase)+(orbital_period_bin*i))-(secondary_eclipse_width)/2 
        eclipse_end = ((orbital_t0_bin+secondary_eclipse_phase)+(orbital_period_bin*i))+(secondary_eclipse_width)/2
        start_secondary.append(eclipse_start)
        end_secondary.append(eclipse_end)
    secondary_limits = np.column_stack([start_secondary,end_secondary])
    mask_limits = np.vstack([mask_limits,secondary_limits]) #masks

mask_time = good_lc.time.value
lc_mask = np.zeros(mask_time.shape, dtype=bool) #creating empty mask

for start, end in mask_limits:
    lc_mask |= (mask_time >= start) & (mask_time <= end) #applying mask for all of limits
masked_lc = good_lc[lc_mask]
print(masked_lc)

#### Check how many points there are
This can indicate whether you should use a random selection (below) or download all (later)

In [ ]:
print(good_lc.info)
####Check the "length" value at the top####

If the length is too big, run the cell below (takes a random sample)

If the length is not too big, run the cell after (takes all data)

#### Save a **random selection** for use in jktebop (less data intensive)

- Set `fraction` to `True` or `False` depending on whether you want to use a fraction (`True`) or or a number of points (`False`)
- Input a float corresponding to the fraction or number 

In [ ]:
####EDITABLE####
fraction = False #If True, take a fraction (0-1) of data points. If False, take a number of data points (e.g. 1000)
value = 1800 #The fraction or number of data points to be used

In [ ]:
#Read in flux
flux = good_lc.flux.value
#Read in flux error 
flux_err = good_lc.flux_err.value
#Calculate magnitude error from flux
Mag_err = (abs((-2.5*np.log10(flux+flux_err))-(-2.5*np.log10(flux)))+abs((-2.5*np.log10(flux))-(-2.5*np.log10(flux-flux_err))))/2
#Calculate magnitude
Mag = -2.5*np.log10(flux)
#Read in time values
time = good_lc.time.value

#Create a new table
jktebop_table = Table()
#Add values to table
jktebop_table['Time'] = time
jktebop_table['Mag'] = Mag
jktebop_table['Mag_err'] = Mag_err

#Write table to txt file
ascii.write(jktebop_table, f'{file_name}_SAMPLE.dat', overwrite=True) 

#Read in table, removing NaN values
test_file = pd.read_csv(f'{file_name}_SAMPLE.dat',delimiter = ' ').dropna()


if fraction is True:
    sample = test_file.sample(frac=value) #look at 50% of the data
else:
    sample = test_file.sample(n=value)

#Upload table to txt in correct format
#Header removed to not confuse jktebop
sample.to_csv(f'{file_name}_SAMPLE.dat',index=False,header=False,sep='\t')

#Printing values needed for use in jktebop parameters
print('Sample Table written!')

print('Orbital period of eclipsing binary system: ',orbital_period)
print('Reference time of primary minimum: ',orbital_t0)
print('Effective Temperature: ',good_lc.meta['TEFF'])

#### Save for use in jktebop **(all points)**

In [ ]:
#Read in flux
flux = good_lc.flux.value
#Read in flux error 
flux_err = good_lc.flux_err.value
#Calculate magnitude error from flux
Mag_err = (abs((-2.5*np.log10(flux+flux_err))-(-2.5*np.log10(flux)))+abs((-2.5*np.log10(flux))-(-2.5*np.log10(flux-flux_err))))/2
#Calculate magnitude
Mag = -2.5*np.log10(flux)
#Read in time values
time = good_lc.time.value

#Create a new table
jktebop_table = Table()
#Add values to table
jktebop_table['Time'] = time
jktebop_table['Mag'] = Mag
jktebop_table['Mag_err'] = Mag_err

#Write table to txt file
ascii.write(jktebop_table, f'{file_name}_ALL.dat', overwrite=True) 

#Read in table, removing NaN values
test_file = pd.read_csv(f'{file_name}_ALL.dat',delimiter = ' ').dropna()

#Upload table to txt in correct format
#Header removed to not confuse jktebop
test_file.to_csv(f'{file_name}_ALL.dat',index=False,header=False,sep='\t')

#Printing values needed for use in jktebop parameters
print('Table written!')

print('Orbital period of eclipsing binary system: ',orbital_period)
print('Reference time of primary minimum: ',orbital_t0)
print('Effective Temperature: ',good_lc.meta['TEFF'])

#### Save Binned Fluxes for Error Estimate
If an error comes with the first line, replace `binned_lc = masked_lc.bin(n_bins = 1000)` with the following:

`binned_lc = masked_lc.bin(time_bin_size=0.01).remove_nans()`

In [ ]:
#####SEE COMMENT ABOVE!!#######
binned_lc = masked_lc.bin(n_bins = 1000) #creating 1000 data points from lightcurve object

#Read in flux
flux = binned_lc.flux.value
#Read in flux error 
flux_err = binned_lc.flux_err.value
#Calculate magnitude error from flux
Mag_err = (abs((-2.5*np.log10(flux+flux_err))-(-2.5*np.log10(flux)))+abs((-2.5*np.log10(flux))-(-2.5*np.log10(flux-flux_err))))/2
#Calculate magnitude
Mag = -2.5*np.log10(flux)
#Read in time values
time = binned_lc.time.value

#Create a new table
jktebop_table = Table()
#Add values to table
jktebop_table['Time'] = time
jktebop_table['Mag'] = Mag
jktebop_table['Mag_err'] = Mag_err

#Write table to txt file
ascii.write(jktebop_table, f'{file_name}_BINNED.dat', overwrite=True) 

#Read in table, removing NaN values
test_file = pd.read_csv(f'{file_name}_BINNED.dat',delimiter = ' ').dropna()

#Upload table to txt in correct format
#Header removed to not confuse jktebop
test_file.to_csv(f'{file_name}_BINNED.dat',index=False,header=False,sep='\t')

#Printing values needed for use in jktebop parameters
print('Table written!')

print('Orbital period of eclipsing binary system: ',orbital_period)
print('Reference time of primary minimum: ',orbital_t0)
print('Effective Temperature: ',good_lc.meta['TEFF'])

#### Saving the edited data of the star as a '.fits' file for later use:

Note the below was written by Pierre Maxted (2024), sent in private communications.

In [ ]:
#####Only run if needed to export as FITS format, otherwise leave#####

#s = lk.search_lightcurve(star_name,author='SPOC',exptime=120)
#lc = s.download_all().stitch()
#Saving edited data as an astropy Table in a FITS file

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    good_lc.write(f'{file_name}.fits',overwrite=True)